In [360]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import datetime as dt
import warnings

warnings.filterwarnings("ignore",category =RuntimeWarning)
pd.options.mode.chained_assignment = None  # default='warn'

#a function to see if something is nan
def is_nan(x):
    return (x is np.nan or x != x)

# Importing the Data

In [361]:
def import_retention_data():
    '''
    A simple function that will read your current working directory and then see
    if there are files for RetentionDataRaw.xlsx and then if they 
    exist will read them in as a pandas data frame
    
    note: if you want to read both files in you must set two variable names equal
    to the function to assign the data frames to names
    '''
    if os.path.exists("data/RetentionDataRaw.xlsx"):
        print("found the retention data")
        data_original = pd.read_excel("data/RetentionDataRaw.xlsx")
        data_original = data_original[:-5963]
        data_original.iloc[:,[5,7,8,9,23]] = data_original.iloc[:,[5,7,8,9,23]].astype(float)
        data_original['External Status'][data_original['External Status'] == " "] = "O"
        
    return data_original

In [362]:
#import the data
data = import_retention_data()

found the retention data


# Editing the Data

## Removing duplicate rows

In [363]:
#create a unique identifier for id number and row number

unique_id_row = []

for i in range(0, len(data)):
    id_num = data['DebtDimId'].iloc[i]
    row_num = data['Row Num'].iloc[i]
    unique_id_row.append(f'{id_num}_{row_num}')
    
data['unique id'] = unique_id_row #add the unique id to the data 
data.drop_duplicates(subset='unique id',inplace=True) #drop duplicate rows
data.drop('unique id', 1, inplace=True)

## Find Customers with no Row 1

In [364]:
unique_id = data['DebtDimId'].unique() # get the unique Id for all of the data
data['Cycle Date'] = pd.to_datetime(data['Cycle Date']) #convert the cycle date to datetime format

no_row_1 = [] #create an empty list

#run a for loop and append the empty list with the id name of customer with no row 1
for ids in unique_id:
    if (data[data['DebtDimId'] == ids]['Row Num'].min() > 1):
        no_row_1.append(ids) 
    
no_row_1_data = data[data['DebtDimId'].isin(no_row_1)] #extract the data from the dataframe
no_row_1_data

,DebtDimId,Open Date,Row Num,Last Statement Date,Cycle Date,Months On Book,External Status,Days Deliq,Credit Limit,Opening Balance,...,Net Cash Advances During Cycle,Net Premier Fees Billed During Cycle,Net Behavior Fees Billed During Cycle,Net Concessions Billed During Cycle,ClosureReason,Month End Date,Last Payment Date,Quarterly Fico Score,Behavior Score,Good Customer Score
10891,10336095.0,2005-06-07,2.0,2010-03-13,2010-04-11,58.0,O,0.0,250.0,0.00,...,0.0,17.0,3.55,11.0,NaN,2010-04-30,2010-04-25,551.0,662.0,738.0
10892,10336095.0,2005-06-07,3.0,2010-04-12,2010-05-12,59.0,O,0.0,250.0,207.75,...,0.0,6.0,3.72,0.0,NaN,2010-05-31,2010-04-25,551.0,658.0,707.0
10893,10336095.0,2005-06-07,4.0,2010-05-13,2010-06-11,60.0,O,0.0,250.0,229.68,...,0.0,54.0,4.02,0.0,NaN,2010-06-30,2010-06-07,551.0,648.0,692.0
10894,10336095.0,2005-06-07,5.0,2010-06-12,2010-07-12,61.0,O,30.0,250.0,278.65,...,0.0,6.0,33.75,0.0,NaN,2010-07-31,2010-06-07,551.0,621.0,678.0
10895,10336095.0,2005-06-07,6.0,2010-07-13,2010-08-12,62.0,O,60.0,250.0,322.66,...,0.0,6.0,34.41,0.0,NaN,2010-08-31,2010-06-07,551.0,580.0,664.0
10896,10336095.0,2005-06-07,7.0,2010-08-13,2010-09-10,63.0,O,91.0,250.0,363.07,...,0.0,6.0,31.09,0.0,NaN,2010-09-30,2010-06-07,551.0,277.0,655.0
10897,10336095.0,2005-06-07,8.0,2010-09-10,2010-10-12,64.0,F,120.0,250.0,400.16,...,0.0,12.0,12.80,0.0,NaN,2010-10-31,2010-06-07,487.0,234.0,NaN
10898,10336095.0,2005-06-07,9.0,2010-10-12,2010-10-28,64.0,Z,151.0,250.0,412.87,...,0.0,6.0,6.71,0.0,NaN,2010-10-31,2010-06-07,487.0,234.0,NaN
48052,18492458.0,2009-01-14,2.0,2010-03-19,2010-04-16,15.0,O,31.0,250.0,0.00,...,0.0,7.0,32.33,0.0,NaN,2010-04-30,2010-03-15,440.0,630.0,NaN
48053,18492458.0,2009-01-14,3.0,2010-04-17,2010-05-18,16.0,O,0.0,250.0,238.07,...,0.0,7.0,3.68,0.0,NaN,2010-05-31,2010-05-11,433.0,646.0,NaN


## Finding Customers with Z in line 1

In [365]:
first_row_Z = data[(data['Row Num'] == 1) & (data['External Status'] == 'Z')] #customers with Z in row 1

first_row_Z

,DebtDimId,Open Date,Row Num,Last Statement Date,Cycle Date,Months On Book,External Status,Days Deliq,Credit Limit,Opening Balance,...,Net Cash Advances During Cycle,Net Premier Fees Billed During Cycle,Net Behavior Fees Billed During Cycle,Net Concessions Billed During Cycle,ClosureReason,Month End Date,Last Payment Date,Quarterly Fico Score,Behavior Score,Good Customer Score
63452,19685699.0,2009-07-17,1.0,2010-02-19,2010-03-04,8.0,Z,31.0,0.0,280.63,...,0.0,-14.0,-29.0,0.0,NaN,2010-03-31,2010-02-11,711.0,558.0,NaN
70957,20156924.0,2009-09-17,1.0,2010-02-20,2010-03-07,6.0,Z,60.0,0.0,327.48,...,0.0,0.0,0.0,0.0,NaN,2010-03-31,2010-01-15,526.0,278.0,NaN
82288,20837262.0,2009-12-09,1.0,2010-02-12,2010-03-05,3.0,Z,60.0,0.0,287.54,...,0.0,0.0,0.0,0.0,NaN,2010-03-31,2009-12-31,513.0,273.0,NaN


## Finding customers that default seeminly at random

In [366]:
over_limit_id = [] #create an empty list

#create a flag for customers who are 90 or more days delinquent
for ids in unique_id:
    over_limit_id.append(1) if data[data['DebtDimId'] == ids]['Days Deliq'].max() >= 90 else over_limit_id.append(0)
    
severe_df = pd.DataFrame({'DebtDimId':unique_id, 'Severe Delinquent': over_limit_id}) #create a key data frame for customers who are 90 or more days delinquent by their ID

data_sevre_del = data.merge(severe_df, on='DebtDimId', how='left') #perform a left join on the original data to add the flag for severe delinquent

In [367]:
id_z = data_sevre_del[data_sevre_del['External Status'] == 'Z']['DebtDimId'].tolist() #get all the IDs for people who default
z_people = data_sevre_del[data_sevre_del['DebtDimId'].isin(id_z)] #get all the data for people who default

In [368]:
z_index = data_sevre_del[data_sevre_del['External Status'] == 'Z'].index #get the index of the observation where people go delinquent
just_z = data_sevre_del[data_sevre_del['External Status'] == 'Z'] #only the data where people go delinquent
z_min_1 = z_index-1 #get the index for delinquent row - 1 to get the credit limit for when they had one


In [369]:
mon_bef_def = data_sevre_del.iloc[z_min_1,:] #get all data for the month before the default

#there are some who still have 0 for credit limit they are listed below after more investigation 
#21169553.0 will be removed based on no reason for them defaulting so soon and 15084368.0.0 will not because they 
#are simply a bad customer
mon_bef_def_0 = mon_bef_def[mon_bef_def['Credit Limit']==0] 
mon_bef_def_non_0 = mon_bef_def[mon_bef_def['Credit Limit']!=0] #these are the credit limits for the month before

id_zero = mon_bef_def_0['DebtDimId'].tolist() #find the id of the 0 month before people
z_without_0 = just_z[~just_z['DebtDimId'].isin(id_zero)] #find id for thosewithout the 0
mon_bef_def_0

,DebtDimId,Open Date,Row Num,Last Statement Date,Cycle Date,Months On Book,External Status,Days Deliq,Credit Limit,Opening Balance,...,Net Premier Fees Billed During Cycle,Net Behavior Fees Billed During Cycle,Net Concessions Billed During Cycle,ClosureReason,Month End Date,Last Payment Date,Quarterly Fico Score,Behavior Score,Good Customer Score,Severe Delinquent
25612,15084368.0,2007-06-24,9.0,2010-10-29,2010-11-28,41.0,C,90.0,0.0,333.14,...,-55.00,36.06,0.0,NaN,2010-11-30,2010-08-16,693.0,247.0,NaN,1
90232,21169553.0,2010-02-01,2.0,2010-03-10,2010-04-08,2.0,C,30.0,0.0,245.26,...,-62.18,28.91,0.0,NaN,2010-04-30,2010-02-28,0.0,279.0,NaN,0


In [370]:
credit_limit_b4_Z = pd.Series(mon_bef_def_non_0['Credit Limit']) #get a series of all credit limits for the customers

z_without_0 = z_without_0.assign(credit_limit_b4_Z=credit_limit_b4_Z.values) #create a column with the credit limit values

#finding the people who are seeminly defaulting at random because they are less than 90 days delinquent,
#they are not overdrawn, and have never been severly delinquent so not a bad customer

second_criteria = z_without_0[(z_without_0['Days Deliq'] < 90) &
                              (z_without_0['Opening Balance'] < z_without_0['credit_limit_b4_Z']) &
                              (z_without_0['Severe Delinquent'] == 0)]

second_criteria_id = second_criteria['DebtDimId'] #get all ids for customers to be removed

In [371]:
#####this was used to remove customers who had closure status that were not delinquent or defaulted but the idea was scraped 

#closed_acc_index = data_sevre_del[data_sevre_del['External Status'] == 'C']['DebtDimId'].unique()
#all_closed_acc = data_sevre_del[data_sevre_del['DebtDimId'].isin(closed_acc_index)]

#close_acc_no_severe = all_closed_acc[all_closed_acc['Severe Delinquent'] == 0]

#no_severe_Z_colsures = close_acc_no_severe[close_acc_no_severe['External Status'] == 'Z']['DebtDimId'].tolist()

#non_bad_closed = close_acc_no_severe[~close_acc_no_severe['DebtDimId'].isin(no_severe_Z_colsures)]

## Remove people who are not in the right cycle of March

In [372]:
data_row_1 = data[data['Row Num'] == 1] #get a data frame with only row 1s

non_march_cycle = data_row_1[(data_row_1['Cycle Date'] > '2010-03-31 23:59:59')]['DebtDimId'].tolist() #get all ids for people whos cycle date is beyond march

## Removing Customers from the Data

In [373]:
remove_list = [] #make an empty list for all ids to be removed

remove_list.extend(first_row_Z['DebtDimId'].tolist())  #remove the Z in first row
remove_list.extend(second_criteria_id.tolist()) #remove based on delinquent at random
remove_list.extend(non_march_cycle) #remove non march cycle
remove_list.extend(no_row_1) #remove no row 1 people
remove_list.extend([21169553.0]) #Remove the one customer discussed above

remove_data = data[data['DebtDimId'].isin(remove_list)] #removed data

#remove_data.to_excel('removed_obs.xlsx') #read to an excel file

## Creating the bad customers

In [374]:
final_customer_all_data = data[~data['DebtDimId'].isin(remove_list)] #customers not in removal list

unique_id_final = final_customer_all_data['DebtDimId'].unique() #unique ids of the customers

bad_id = []

#create the bad customers based on if they were ever 90 or more days delinquent or they ever defaulted in the 10 months
for ids in unique_id_final:
    bad_id.append(1) if final_customer_all_data[final_customer_all_data['DebtDimId'] ==
                                                ids]['Days Deliq'].max() >= 90 or final_customer_all_data[final_customer_all_data['DebtDimId']  ==
                                                                                                         ids]['External Status'].isin(['Z']).sum() > 0 else bad_id.append(0)

## Create the dataframes of bad customers

In [375]:
bad_df = pd.DataFrame({'DebtDimId':unique_id_final, 'Bad': bad_id}) #make a index data frame of bad customers

final_cust_data_bad = final_customer_all_data.merge(bad_df, on='DebtDimId', how='left') #create a data frame by left joining on the final customer data frame
row_1_final_cust = final_cust_data_bad[final_cust_data_bad['Row Num'] == 1] #create the actual data set with only row 1

In [376]:
###This was done to see what percentage of people who were ever 90 days delinquent actually defaulted

cust_over_90 = []

for ids in unique_id_final:
    if final_customer_all_data[final_customer_all_data['DebtDimId'] == ids]['Days Deliq'].max() >= 90:
        cust_over_90.append(ids)
        
cust_over_90_Z = []

for ids in unique_id_final:
    if final_customer_all_data[final_customer_all_data['DebtDimId'] == ids]['Days Deliq'].max() >= 90 and final_customer_all_data[final_customer_all_data['DebtDimId']  ==
                                                                                                         ids]['External Status'].isin(['Z']).sum() > 0:
        cust_over_90_Z.append(ids)
        

len(cust_over_90_Z)/len(cust_over_90)

0.6908948808128175

# Create Custom Input Variables

In [377]:
payment_type = []

#creating an index of whether someone payed above, below, exactly their minimum payment or if they didnt pay at all

for i in range(0, len(row_1_final_cust)):
    if (row_1_final_cust['Net Payments During Cycle'].iloc[i]>row_1_final_cust['Total Min Pay Due'].iloc[i]):
        payment_type.append('Above')
    elif (row_1_final_cust['Net Payments During Cycle'].iloc[i]==row_1_final_cust['Total Min Pay Due'].iloc[i]):
        payment_type.append('Exact')
    elif (row_1_final_cust['Net Payments During Cycle'].iloc[i]==0):
        payment_type.append('No Payment')
    else:
        payment_type.append('Below')
        
        
closure_reas_pres = []

#creating an index of whether there is a closure reason given

for i in range(0, len(row_1_final_cust)):
    if (is_nan(row_1_final_cust['ClosureReason'].iloc[i])):
        closure_reas_pres.append(0)
    else:
        closure_reas_pres.append(1)

deliq_and_over = []

#creating an index of whether a customer is both delinquent and over their limit

for i in range(0, len(row_1_final_cust)):
    if (row_1_final_cust['Days Deliq'].iloc[i] > 0 and row_1_final_cust['Over limit Amount'].iloc[i] > 0):
        deliq_and_over.append(1)
    else:
        deliq_and_over.append(0)

In [378]:
row_1_final_cust['Beginning Utilization'] = row_1_final_cust['Opening Balance']/row_1_final_cust['Credit Limit'] #opening month card utilization
row_1_final_cust['Ending Utilization'] = row_1_final_cust['Ending Balance']/row_1_final_cust['Credit Limit'] #end of month utilization
row_1_final_cust['Utilization Difference'] = row_1_final_cust['Ending Utilization']-row_1_final_cust['Beginning Utilization'] #how much the utilization has changed from the beginning to end of the month
row_1_final_cust['Total Fees and Concessions'] = row_1_final_cust['Net Premier Fees Billed During Cycle'] + row_1_final_cust['Net Behavior Fees Billed During Cycle'] - row_1_final_cust['Net Concessions Billed During Cycle']
row_1_final_cust['Payment Type'] = payment_type
row_1_final_cust['Closure Reason Precense'] = closure_reas_pres
row_1_final_cust['Delinquent and Overlimit'] = deliq_and_over

# Summary Outcomes

In [379]:
#We are creating a summary outcome for net purchases total for all months where the account isnt closed, frozen or defaulted

id_final = final_cust_data_bad['DebtDimId'].unique()

Net_Purchases_total = []

for ids in id_final:
     Net_Purchases_total.append(final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['Net Purchases During Cycle'].sum())

row_1_final_cust['Net Purchases Total'] = Net_Purchases_total

In [380]:
#We are calculating profits by summing up net payments for accounts that are not 'bad' and taking the ending balance or the second to last ending balance for
#'bad' customers and saying its a negative profit

Profits = []

for ids in id_final:
    if final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['Bad'].sum() == 0:
        Profits.append(final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['Net Payments During Cycle'].sum())
    else:
        if final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['External Status'].iloc[-1] == 'Z':
             Profits.append(-final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['Ending Balance'].iloc[-2])
        else:
             Profits.append(-final_cust_data_bad[final_cust_data_bad['DebtDimId'] == ids]['Ending Balance'].iloc[-1])
        
row_1_final_cust['Profit Per Account'] = Profits

# Read the data to Excel File

In [381]:
with pd.ExcelWriter('final_model_data.xlsx') as writer:
    final_cust_data_bad.to_excel(writer, sheet_name='All Observations')
    row_1_final_cust.to_excel(writer, sheet_name='Row 1 Observations')